# Initializing your environment

### Setting up a virtual environment
(Optional but recommended)

```bash
virtualenv -p python3 venv
source venv/bin/activate
```
(Use `deactivate` to exit from `source` once you are done)

Alternatively you can also prefix all your `python` and `pip` commands with `./venv/bin/` (e.g: `./venv/bin/pip3 install -U pip`)

## Setting up Jupyter

In order to follow along on your computer:

```bash
pip3 install notebook
jupyter-notebook
```

## Installation of PyTaxonomies

#### Make sure the submodules are up-to-date and cloned

```bash
git submodule update --init --recursive PyTaxonomies/
```

#### Installation

```bash
cd PyTaxonomies
python3 setup.py install
```

# Basics

## Taxonomies basic information

In [ ]:
from pytaxonomies import Taxonomies

taxonomies = Taxonomies()
print(f'Version: {taxonomies.version}')
print(f'License: {taxonomies.license}')
print(f'Description: {taxonomies.description}')
print(f'Number of taxonomies: {len(taxonomies)}')
print(f'Names: {list(taxonomies.keys())}')

## Get information on a specific taxonomy

In [ ]:
# Pick whichever taxonomy name
namespace = 'honeypot-basic'

### Standard information on the taxonomy

In [ ]:
taxonomy = taxonomies.get(namespace)
for feature in ('name', 'version', 'description'):
    print(f'{feature}: {getattr(taxonomy, feature)}')
print()
print(f'Number of predicates: {len(taxonomy)}')
print(f'Number of entries: {taxonomy.amount_entries()}')

### Overview of the predicates

In [ ]:
print(f'predicates: {list(taxonomy.keys())}\n')

for predicate, values in taxonomy.predicates.items():
    if len(values.keys()) == 0:
        print(predicate)
    else:
        print(f'{predicate}:')
        for value in values:
            print(f' - {value}')

### Detailed information on the predicates and their values

In [ ]:
# Pick wichever of the predicates displayed above
predicate_name = 'data-capture'
predicate = taxonomy.get(predicate_name)

In [ ]:
# In the case of TLP & PAP, there are only predicates
if len(predicate.keys()) == 0:
    for feature in ('predicate', 'expanded', 'description'):
        print(f'{feature}: {getattr(predicate, feature)}')
else:
    for value in predicate.keys():
        print(f'{value}:')
        for feature in ('value', 'expanded', 'description'):
            print(f' - {feature}: {getattr(predicate.get(value), feature)}')

#### And the easiest way

In [ ]:
# It will fail if there are only predicates and no values

value = 'network-capture'
print(f'value: {predicate.get(value).value}')
print(f'expanded value: {predicate.get(value).expanded}')
print(f'description: {predicate.get(value).description}')
print()

# or directly
print(taxonomies.get(namespace).get(predicate_name).get(value).value)

### Display the tags

In [ ]:
print(taxonomy.machinetags())
print(f'\nAnd with the expanded values:\n{taxonomy.machinetags_expanded()}')

# Create a new taxonomy

## Declare a new Taxonomy

In [ ]:
from pytaxonomies import Taxonomy

new_taxonomy = Taxonomy()

new_taxonomy.name = "false-positive"
new_taxonomy.description = "This taxonomy aims to ballpark the expected amount of false positives."
new_taxonomy.version = 1
new_taxonomy.expanded = "False positive"

print(new_taxonomy.to_json())

## Declare the predicates

In [ ]:
from pytaxonomies import Predicate

risk_predicate = Predicate()
risk_predicate.predicate = 'risk'
risk_predicate.expanded = 'Risk'
risk_predicate.description = 'Risk of having false positives in the tagged value.'
risk_predicate.exclusive = True
print(risk_predicate.to_json())

confirmed_predicate = Predicate()
confirmed_predicate.predicate = 'confirmed'
confirmed_predicate.expanded = 'Confirmed'
confirmed_predicate.description = 'Confirmed false positives in the tagged value.'
confirmed_predicate.exclusive = True
print(confirmed_predicate.to_json())

## Declare the entries

In [ ]:
from pytaxonomies import Entry

# Entries for the 'risk' predicate
low = Entry()
low.value = 'low'
low.expanded = 'Low'
low.description = 'The risk of having false positives in the tagged value is low.'
low.numerical_value = 20

medium = Entry()
medium.value = 'medium'
medium.expanded = 'Medium'
medium.description = 'The risk of having false positives in the tagged value is medium.'
medium.numerical_value = 50

high = Entry()
high.value = 'high'
high.expanded = 'High'
high.description = 'The risk of having false positives in the tagged value is high.'
high.numerical_value = 75

cannot_be_judged = Entry()
cannot_be_judged.value = 'cannot-be-judged'
cannot_be_judged.expanded = 'Risk cannot be judged'
cannot_be_judged.description = 'The risk of having false positives in the tagged value cannot be judged.'
cannot_be_judged.numerical_value = 25

# Entries for the 'confirmed' predicate
true = Entry()
true.value = 'true'
true.expanded = 'True'
true.description = 'The false positive is confirmed.'
true.numerical_value = 100

false = Entry()
false.value = 'false'
false.expanded = 'False'
false.description = 'The false positive is not confirmed.'
false.numerical_value = 0

## Add the predicates and values

In [ ]:
risk_predicate.entries = {
    'low': low,
    'medium': medium,
    'high': high,
    'cannot-be-judged': cannot_be_judged
}

confirmed_predicate.entries = {
    'true': true, 'false': false
}

new_taxonomy.predicates = {
    'risk': risk_predicate,
    'confirmed': confirmed_predicate
}

In [ ]:
print(new_taxonomy.machinetags())

## Add the new taxonomy in the repository

In [ ]:
import json
from pathlib import Path

taxonomies_path = Path().resolve().parent / 'PyTaxonomies' / 'pytaxonomies' / 'data' / 'misp-taxonomies'

with open(taxonomies_path / 'MANIFEST.json', 'rt', encoding='utf-8') as m:
    manifest = json.load(m)
    
# We check if the taxonomy already exists.
is_new = True
for taxonomy in manifest['taxonomies']:
    if taxonomy['name'] == new_taxonomy.name:
        # The taxonomy already exists, we update it
        taxonomy['version'] += 1
        taxonomy['description'] = new_taxonomy.description
        is_new = False
        
if is_new:
    # It is a new taxonomy, we add it in the manifest
    manifest['taxonomies'].append(
        {
            'version': new_taxonomy.version,
            'name': new_taxonomy.name,
            'description': new_taxonomy.description
        }
    )
    
with open(taxonomies_path / 'MANIFEST.json', 'wt', encoding='utf-8') as f:
    f.write(json.dumps(manifest, indent=2, ensure_ascii=False))
    
if not (taxonomies_path / new_taxonomy.name).exists():
    (taxonomies_path / new_taxonomy.name).mkdir()
    
with open(taxonomies_path / new_taxonomy.name / 'machinetag.json', 'wt', encoding='utf-8') as f:
    json.dump(new_taxonomy.to_dict(), f, indent=2, ensure_ascii=False)

# Edit a taxonomy

In [ ]:
from pytaxonomies import Taxonomies

taxonomies = Taxonomies()

edited_taxonomy = taxonomies['false-positive']

edited_taxonomy.predicates['risk'].entries['low'].numerical_value = 25

## Save the edited taxonomy

In [ ]:
import json
from pathlib import Path

taxonomies_path = Path().resolve().parent / 'pytaxonomies' / 'data' / 'misp-taxonomies'

with open(taxonomies_path / 'MANIFEST.json', 'rt', encoding='utf-8') as m:
    manifest = json.load(m)
    
# We check if the taxonomy already exists.
exists = False
for taxonomy in manifest['taxonomies']:
    if taxonomy['name'] == edited_taxonomy.name:
        taxonomy['version'] += 1
        edited_taxonomy.version = taxonomy['version']
        taxonomy['description'] = edited_taxonomy.description
        exists = True
        
if not exists:
    raise Exception(f'The {edited_taxonomy.name} Taxonomy does not exist in the manifest.')
    
with open(taxonomies_path / 'MANIFEST.json', 'w', encoding='utf-8') as f:
    json.dump(manifest, f, indent=2, ensure_ascii=False)
    
with open(taxonomies_path / edited_taxonomy.name / 'machinetag.json', 'w', encoding='utf-8') as f:
    json.dump(edited_taxonomy.to_dict(), f, indent=2, ensure_ascii=False)